In [1]:
import os
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.optim import Adam
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from torchmetrics import MeanAbsolutePercentageError


from data.dataloader import dataloader_info
from utils.utils import load_yaml_config, instantiate_from_config

from models.predictor import GRU
from models.solver import Trainer
from data.dataloader import dataloader_info
from utils.visualize import visualize_pca, visualize_tsne, visualize_kernel

/home/harim/Desktop/pyproject/Project_TS_Generation/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Training DiffusionTS With Differencing

In [2]:
# Load configurations
configs = load_yaml_config("configs/experiments1_w_diff.yaml")
device = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize Diffusion_TS Model
diffusion_ts = instantiate_from_config(configs['model']).to(device)
batch_size = 128

In [3]:
# load dataloader, dataset
dl_info_train = dataloader_info(configs, train=True)
dl_info_test = dataloader_info(configs, train=False)

dl_train = dl_info_train["dataloader"]
ds_train = dl_info_train["dataset"]

dl_test = dl_info_test["dataloader"]
ds_test = dl_info_test["dataset"]


In [4]:
trainer = Trainer(config_solver=configs["solver"], model=diffusion_ts, dataloader=dl_train)
trainer.train()

  0%|          | 0/20000 [00:00<?, ?it/s]/home/harim/Desktop/pyproject/Project_TS_Generation/.venv/lib/python3.12/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
combiend_loss: 0.034673 l1_loss : 0.394106 fourier_loss : 0.000000:  50%|█████     | 10084/20000 [14:31<14:17, 11.57it/s]


KeyboardInterrupt: 

# Training DiffusionTS WithOut Differencing

In [2]:
# Load configurations
configs = load_yaml_config("configs/experiments1_wo_diff.yaml")
device = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize Diffusion_TS Model
diffusion_ts = instantiate_from_config(configs['model']).to(device)
batch_size = 128

In [3]:
# load dataloader, dataset
dl_info_train = dataloader_info(configs, train=True)
dl_info_test = dataloader_info(configs, train=False)

dl_train = dl_info_train["dataloader"]
ds_train = dl_info_train["dataset"]

dl_test = dl_info_test["dataloader"]
ds_test = dl_info_test["dataset"]


In [4]:
trainer = Trainer(config_solver=configs["solver"], model=diffusion_ts, dataloader=dl_train)
trainer.train()

  0%|          | 0/10000 [00:00<?, ?it/s]/home/harim/Desktop/pyproject/Project_TS_Generation/.venv/lib/python3.12/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
combiend_loss: 0.045692 l1_loss : 0.598499 fourier_loss : 0.000000: 100%|██████████| 10000/10000 [14:40<00:00, 11.35it/s]


In [5]:
fake_data = diffusion_ts.generate_mts(batch_size=3000)
np.save("dataset/stock/fake_e10000_n3000", fake_data)

reverse step from x_T to x_0:   0%|          | 0/100 [00:00<?, ?it/s]/home/harim/Desktop/pyproject/Project_TS_Generation/.venv/lib/python3.12/site-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,
reverse step from x_T to x_0:  11%|█         | 11/100 [00:07<01:04,  1.38it/s]


KeyboardInterrupt: 

In [ ]:
# load original data
idx = np.random.permutation(len(ds_train.data_diff))[:3000]
ori_data = ds_train.data_diff[idx]

In [ ]:
visualize_pca(ori_data, fake_data, 3000)
visualize_tsne(ori_data, fake_data, 3000)
visualize_kernel(ori_data, fake_data, 3000)